In [18]:
import pandas as pd
import numpy as np
import requests
import json
import pickle
import collections
from bs4 import BeautifulSoup
import multiprocessing
from time import sleep
from Levenshtein import ratio

In [2]:
BB = pd.read_csv("../data/Hot Stuff.csv")
songs = BB["SongID"]; performer = BB["Performer"]
song_performer_pair = list(BB.groupby(["Performer", "Song"]).count().index)
len(song_performer_pair)

27949

In [3]:
client_id = "HYxx-SDO43-ap9RUVgD9nNRuz-kmhQw9W-TeCtA4NdmWbLYQUvkCmmcOlNzsdzRE"
client_secred = "2xRkeAd2SDvMLXbkScUzLCZvOPxYXcsdQeiDqhrax-kzxpFkO3fKI4HEKeD5aDlXnYnja7pwYQ9BkMoDvl6ipw"
token = "dxvLAG0Jhx6B01DK9pLzusCIFlboFV996sOvwEAPQbON-MUPz2vLgu9_z7u-hODo"
headers = {'Authorization': "Bearer " + token}
base_url = "http://api.genius.com"
search_url = base_url + "/search"

In [4]:
def reformat_aritst_song(artist, song):
    if "Featuring" in artist:
        artist = artist.split("Featuring")[0].strip()
    if "Feat." in artist:
        artist = artist.split("Feat.")[0].strip()
    if "featuring" in artist:
        artist = artist.split("featuring")[0].strip()
    if "&" in artist:
        artist = artist.split("&")[0].strip()
    if "and" in artist:
        artist = artist.split("and")[0].strip()
    if "And" in artist:
        artist = artist.split("And")[0].strip()
    if "With" in artist:
        artist = artist.split("With")[0].strip()
    if "with" in artist:
        artist = artist.split("with")[0].strip()
    if "The" in artist:
        artist = artist.split("The")[0].strip()
    if "the" in artist:
        artist = artist.split("the")[0].strip()
    if "(" in artist:
        artist = artist.split("(")[0].strip()
    return [artist, song]

In [64]:
song_title = "Hey, Little Girl"
artist_name = 'Foster Sylvers'
params = {'q': song_title + " by " + artist_name}
response = requests.get(search_url, params=params, headers=headers).json()

In [56]:
searchurl = "https://genius.com/Foster-sylvers-hey-little-girl-lyrics"
response = requests.get(searchurl)

In [57]:
html = BeautifulSoup(response.text, "html.parser")

In [63]:
html.find("div", class_="lyrics)

SyntaxError: EOL while scanning string literal (<ipython-input-63-02e6b17f29b2>, line 1)

In [40]:
for hit in response["response"]["hits"]:
    print("title:  {},    artist:  {}".format(hit["result"]["title"], hit["result"]["primary_artist"]["name"]))

title:  Slow Down,    artist:  Krizz Kaliko
title:  Home Alone,    artist:  John Hughes
title:  Brave New World (Chapter 3),    artist:  Aldous Huxley
title:  Pulp Fiction full screenplay,    artist:  Quentin Tarantino
title:  Boogie Nights Screenplay,    artist:  Paul Thomas Anderson
title:  Winter Dreams,    artist:  F. Scott Fitzgerald
title:  Bliss.,    artist:  Katherine Mansfield
title:  Wonder Tales from Scottish Myth and Legend - Chapter XIV: Exiles from Fairyland,    artist:  Donald Alexander Mackenzie
title:  Week 7 Reading Journal,    artist:  Iris Venture-Ayala


In [10]:
def get_lyrics(i):
#     artist_name, song_title = song_performer_pair[i]
    artist_name, song_title = song_pair_missing[i]
    artist_name_reformatted, song_title_reformatted = artist_name, song_title
    params = {'q': song_title.lower() + " " + artist_name.lower()}
#     params = {'q': song_title.lower()}
    try:
        if i%1000 == 0:
            print(i)
        response = requests.get(search_url, params=params, headers=headers).json()
        song_info = False
        for hit in response["response"]["hits"]:
            if len(set(hit["result"]["primary_artist"]["name"].lower().split(" ")).intersection(set(artist_name.lower().split(" ")))) != 0:
                song_info = hit
                break
        if not song_info:
            artist_name_reformatted, song_title = reformat_aritst_song(artist_name, song_title)
            params = {'q': song_title.lower() + " " + artist_name_reformatted.lower()}
            response = requests.get(search_url, params=params, headers=headers).json()
            for hit in response["response"]["hits"]:
                if len(set(hit["result"]["primary_artist"]["name"].lower().split(" ")).intersection(set(artist_name.lower().split(" ")))) != 0:
                    song_info = hit
                    break
        if song_info:
            song_api_path = song_info["result"]["api_path"]
            song_url = base_url + song_api_path
            response = requests.get(song_url, headers=headers).json()
            path = response["response"]["song"]["path"]
            page_url = "http://genius.com" + path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, "html.parser")
            [h.extract() for h in html('script')]
            lyrics = html.find("div", class_="lyrics").get_text()
            return [artist_name, song_title, [i for i in lyrics.split("\n") if i != ""], page_url]
        else:
            return [False, artist_name, song_title, None]
    except Exception as e:
        print(e)
        print("Site thinks im tryna DDOS, lol")
        sleep(np.random.random()*10)
        return get_lyrics(i)

In [6]:
art_song_lyrics = collections.defaultdict(dict)
missing_artists_song = collections.defaultdict(set)
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*16) as pool:
    try:
        for suc,val1,val2 in pool.imap_unordered(get_lyrics, range(len(song_performer_pair))):
            if type(suc) == str:
                art_song_lyrics[suc][val1] = val2
            else:
                missing_artists_song[val1].add(val2)
    except Exception as e:
        print(e)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
Expecting value: line 1 column 1 (char 0)
Site thinks im tryna DDOS, lol
25000
26000
27000


In [6]:
missing_artists_song = pickle.loads(open("../data/pickle/Genius_missing4.pickle", "rb").read())
art_song_lyrics = pickle.loads(open("../data/pickle/Genius_found4.pickle", "rb").read()) 

In [7]:
temp = {}
for i in list(missing_artists_song.keys()):
    temp[i] = list(missing_artists_song[i])

In [8]:
Genius_artist_song_pair = [[i, j] for i in art_song_lyrics.keys() for j in art_song_lyrics[i].keys() ]
song_pair_missing = [[i, j] for i in list(missing_artists_song.keys()) for j in list(missing_artists_song[i])]
print("No. Found Song Pair:  {}".format(len(Genius_artist_song_pair)))
print("No. Missing Song Pair:  {}".format(len(song_pair_missing)))
assert(len(Genius_artist_song_pair) + len(song_pair_missing) == len(song_performer_pair))

No. Found Song Pair:  24534
No. Missing Song Pair:  3415


In [15]:
# pickle.dump(art_song_lyrics, open("../data/pickle/Genius_found4.pickle", "wb"))
# pickle.dump(temp, open("../data/pickle/Genius_missing4.pickle", "wb"))

In [14]:
# jdata = json.dumps(temp)
# output_file = open("../data/json/Genius_missing2.json","w+")
# output_file.write(jdata)
# output_file.close()

# jdata = json.dumps(art_song_lyrics)
# output_file = open("../data/json/Genius_data2.json","w+")
# output_file.write(jdata)
# output_file.close()

In [9]:
collections.Counter([j for i in list(missing_artists_song.keys()) for j in i.split(" ")]).most_common(20)

[('The', 313),
 ('&', 189),
 ('Featuring', 113),
 ('And', 102),
 ('His', 48),
 ('Orchestra', 46),
 ('and', 42),
 ('Bobby', 38),
 ('Band', 37),
 ('With', 32),
 ('Johnny', 30),
 ('Joe', 28),
 ('Billy', 26),
 ('Of', 23),
 ('Lee', 23),
 ('Jimmy', 22),
 ('Brown', 21),
 ('Paul', 19),
 ('Ray', 18),
 ('John', 17)]

In [10]:
missing_artists_song_old = dict(missing_artists_song)

In [11]:
missing_artists_song = collections.defaultdict(set)
new_added = collections.defaultdict(dict)
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*16) as pool:
    try:
        for suc,val1,val2, val3 in pool.imap_unordered(get_lyrics, range(len(song_pair_missing))):
            if type(suc) == str:
                art_song_lyrics[suc][val1] = val2
                new_added[suc][val1] = {"lyrics":  val2, "url": val3}
            else:
                missing_artists_song[val1].add(val2)
    except Exception as e:
        print(e)

0
1000
2000
3000


In [12]:
song_pair_missing_2 = [[i, j] for i in list(missing_artists_song.keys()) for j in list(missing_artists_song[i])]
print(len(song_pair_missing_2))

3409


In [4]:
genius_data = json.loads(open('./Genesis_data.json').read())

In [5]:
lyric_l = []
genius_data['"Weird Al" Yankovic']
for art in genius_data:
    for song in genius_data[art]:
        lyrics = "\n".join(genius_data[art][song])
        lyric_l.append([art,song,lyrics])

In [6]:
genius_df = pd.DataFrame(data=lyric_l,columns=["Artist","Song","Lyrics"])
genius_df.head()

,Artist,Song,Lyrics
0,"""Weird Al"" Yankovic",Canadian Idiot,[Verse 1]\nDon't wanna be a Canadian idiot\nDo...
1,"""Weird Al"" Yankovic",Like A Surgeon,I finally made it through med school\nSomehow ...
2,"""Weird Al"" Yankovic",Amish Paradise,[Verse 1]\nAs I walk through the valley where ...
3,"""Weird Al"" Yankovic",Eat It,[Verse 1:]\nHow come you're always such a fuss...
4,"""Weird Al"" Yankovic",Ricky,"[Ricky]\nHey Lucy, I'm home\n[Lucy]\nOh Ricky,..."


In [13]:
genius_df.shape

(23454, 3)

In [7]:
import re
list_of_words = lambda ly : " ".join(re.split(r"\[[\s\S]*?\]|[^\w'\\-]|\s",ly.lower())).split()
from collections import Counter
bags = [Counter(list_of_words(ly)) for ly in genius_df["Lyrics"]]

In [8]:
lists = [list_of_words(ly) for ly in genius_df["Lyrics"]]

In [9]:
genius_df["Bag"] = bags
genius_df["List"] = lists

In [10]:
genius_df.head()

,Artist,Song,Lyrics,Bag,List
0,"""Weird Al"" Yankovic",Canadian Idiot,[Verse 1]\nDon't wanna be a Canadian idiot\nDo...,"{'don't': 3, 'wanna': 3, 'be': 3, 'a': 7, 'can...","[don't, wanna, be, a, canadian, idiot, don't, ..."
1,"""Weird Al"" Yankovic",Like A Surgeon,I finally made it through med school\nSomehow ...,"{'i': 7, 'finally': 1, 'made': 2, 'it': 2, 'th...","[i, finally, made, it, through, med, school, s..."
2,"""Weird Al"" Yankovic",Amish Paradise,[Verse 1]\nAs I walk through the valley where ...,"{'as': 5, 'i': 14, 'walk': 1, 'through': 1, 't...","[as, i, walk, through, the, valley, where, i, ..."
3,"""Weird Al"" Yankovic",Eat It,[Verse 1:]\nHow come you're always such a fuss...,"{'how': 1, 'come': 1, 'you're': 5, 'always': 1...","[how, come, you're, always, such, a, fussy, yo..."
4,"""Weird Al"" Yankovic",Ricky,"[Ricky]\nHey Lucy, I'm home\n[Lucy]\nOh Ricky,...","{'hey': 10, 'lucy': 17, 'i'm': 5, 'home': 1, '...","[hey, lucy, i'm, home, oh, ricky, you're, so, ..."


In [11]:
from collections import defaultdict
big_word_bag = defaultdict(int)
for ls in genius_df["List"]:
    for word in ls:
        big_word_bag[word]+=1
len(big_word_bag)

153645

In [14]:
sorted_words = sorted(big_word_bag,key=lambda w: big_word_bag[w],reverse=True)

In [15]:
for i in range(20):
    print(i,sorted_words[i],big_word_bag[])

0 the 0
1 i 0
2 you 0
3 and 0
4 to 0
5 a 0
6 of 0
7 in 0
8 it 0
9 me 0
10 that 0
11 my 0
12 on 0
13 is 0
14 for 0
15 be 0
16 your 0
17 with 0
18 all 0
19 i'm 0
